In [1]:
import sys
sys.path.append('../src')
import pre_process as pre
import pre_process_demo_helper_fcns as pre_demo
import solver_demo_helper_functions as solver_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline


In [2]:
# Geometry and mesh
L = 10.0
H = 1.0
nx = 20
ny = 2
ele_type = "D2_nn8_quad"
ndof = 2

coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)


In [3]:
# Boundary conditions and large load setup
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(
    coords, connect, ele_type, x_lower=0.0, x_upper=L, y_lower=0.0, y_upper=H
)

fixed_nodes = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)
q_large = -100.0  # Large load to force non-convergence
dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, q_large)


In [4]:
# Very soft material and nearly incompressible behavior
E_soft = 100.0
nu = 0.49
mu = E_soft / (2 * (1 + nu))
kappa = E_soft / (3 * (1 - 2 * nu))
material_props = np.array([mu, kappa])


In [6]:
# Run the solver and expect matrix rank warning
try:
    displacements_all, _ = hyperelastic_solver(
        material_props,
        ele_type,
        coords.T,
        connect.T,
        fixed_nodes,
        dload_info,
        nr_print=True,
        nr_num_steps=1,
        nr_tol=1e-8,
        nr_maxit=30
    )
    print("Simulation completed successfully")
except Exception as e:
    print("Simulation failed to converge")
    print("Reason:", str(e))


Step 0, load factor = 1.000
Iteration 1, Correction=1.000000e+00, Residual=5.031530e-01, tolerance=1.000000e-08
Iteration 2, Correction=0.000000e+00, Residual=nan, tolerance=1.000000e-08
Simulation completed successfully


In [7]:
# === Convergence Diagnostic Summary ===
if displacements_all is not None:
    final_disp = displacements_all[-1]
    has_nan = np.isnan(final_disp).any()
    correction_nan = False

    print("\n===== CONVERGENCE SUMMARY =====")
    if has_nan:
        print("Displacement vector contains NaNs — numerical divergence occurred.")
    else:
        print("Displacement vector is valid, but review residual/correction values carefully.")

    # You can optionally check for NaNs in the last residual (if accessible from your solver)
    print("Note: Residual printed during solver run showed 'nan' — Newton iteration likely diverged.")
else:
    print("Solver did not return displacements — hard failure occurred.")



===== CONVERGENCE SUMMARY =====
Displacement vector contains NaNs — numerical divergence occurred.
Note: Residual printed during solver run showed 'nan' — Newton iteration likely diverged.


In [8]:
# Explanation of failure
print("""
WHY DID THIS FAIL?
------------------
- Material is extremely soft (E = 100), making the system highly nonlinear.
- Load is applied abruptly in a single step, with no ramping.
- The Newton-Raphson method assumes small incremental corrections — this setup breaks that assumption.
- Nearly incompressible material (ν ≈ 0.49) leads to a poorly conditioned stiffness matrix.
- This can cause:
    - NaNs in the residual or displacement vector
    - Matrix singularity warnings
    - Solver stagnation or divergence without throwing a formal error
""")



WHY DID THIS FAIL?
------------------
- Material is extremely soft (E = 100), making the system highly nonlinear.
- Load is applied abruptly in a single step, with no ramping.
- The Newton-Raphson method assumes small incremental corrections — this setup breaks that assumption.
- Nearly incompressible material (ν ≈ 0.49) leads to a poorly conditioned stiffness matrix.
- This can cause:
    - NaNs in the residual or displacement vector
    - Matrix singularity warnings
    - Solver stagnation or divergence without throwing a formal error

